# Duplicate Data

A data set might have duplicate data: in other words, the same record is represented multiple times. Sometimes, it's easy to find and eliminate duplicate data like when two records are exactly the same. At other times, like what was discussed in the video, duplicate data is hard to spot. 

# Exercise 1

From the World Bank GDP data, count the number of countries that have had a project totalamt greater than 1 billion dollars (1,000,000,000). To get the count, you'll have to remove duplicate data rows.

In [1]:
import pandas as pd
import datetime 
import numpy as np

In [2]:
# read in the projects data set and do some basic wrangling 
projects = pd.read_csv('./data/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# TODO: filter the data frame for projects over 1 billion dollars
# TODO: count the number of unique countries in the results

In [3]:
projects.query('totalamt > 1000000000').shape

(44, 56)

In [4]:
projects.query('totalamt > 1000000000')['countryname'].nunique()

17

In [5]:
projects.query('totalamt > 1000000000').duplicated(subset=['countryname']).sum()

27

In [6]:
projects.query('totalamt > 1000000000').drop_duplicates(subset=['countryname']).shape[0]

17

# Exercise 2 (challenge)

This exercise is more challenging. The projects data set contains data about Yugoslavia, which was an Eastern European country until 1992. Yugoslavia eventually broke up into 7 countries: Bosnia and Herzegovina, Croatia, Kosovo, Macedonia, Montenegro, Serbia, and Slovenia.

But the projects dataset has some ambiguity in how it treats Yugoslavia and the 7 countries that came from Yugoslavia. Your task is to find Yugoslavia projects that are probably represented multiple times in the data set.

In [7]:
# Official name is 'Socialist Federal Republic of Yugoslavia' 
# with possibility of variation

yugoslavia = projects[projects.countryname.str.contains('Yugo')]
yugoslavia.shape

(90, 56)

Yugoslavia officially ended on [April 27th, 1992](https://en.wikipedia.org/wiki/Yugoslavia). 

In the code cell below, filter for projects with a 'boardapprovaldate' prior to April 27th, 1992 **and** with 'countryname' Bosnia and Herzegovina, Croatia, Kosovo, Macedonia, Serbia **or** Slovenia. Save the results in the republics variable

In [8]:
republics_list = ['Bosnia and Herzegovina', 'Croatia', 'Kosovo', 'Macedonia', 
                  'Serbia', 'Montenegro', 'Slovenia']

cols_to_keep = ['regionname', 'countryname', 'lendinginstr', 'totalamt',
                'boardapprovaldate','location', 'GeoLocID', 'GeoLocName',
                'Latitude', 'Longitude', 'Country', 'project_name']

projects = projects[cols_to_keep]

republics = projects[
    (projects.boardapprovaldate <= '1992-04-27') &
    (projects.countryname.isin(republics_list))
]

In [9]:
republics 

,regionname,countryname,lendinginstr,totalamt,boardapprovaldate,location,GeoLocID,GeoLocName,Latitude,Longitude,Country,project_name
12063,Europe and Central Asia,Bosnia and Herzegovina,Sector Investment and Maintenance Loan,0,1987-10-13 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,HIGHWAY SECTOR II
13048,Europe and Central Asia,Bosnia and Herzegovina,Specific Investment Loan,0,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,POWER TRANS.III
13050,Europe and Central Asia,Macedonia,Specific Investment Loan,0,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,POWER TRANS.III
13973,Europe and Central Asia,Macedonia,Specific Investment Loan,24000000,1980-02-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,Agriculture & Agroindustry 2 Project (Macedonia)


Are these projects also represented in the data labeled Yugoslavia? In the code cell below, filter for Yugoslavia projects approved between February 1st, 1980 and May 23rd, 1989 which are the minimum and maximum dates in the results above. Store the results in the yugoslavia variable.

The goal is to see if there are any projects represented more than once in the data set.

In [15]:
yugoslavia = projects[projects.countryname.str.contains('Yugo')]
yugoslavia = yugoslavia[(yugoslavia.boardapprovaldate >= '1980-2-1') & 
                        (yugoslavia.boardapprovaldate <= '1989-5-23')]

yugoslavia.shape

(31, 12)

In [19]:
republic_unique_dates = [republics.boardapprovaldate.sort_values().unique()]
yugoslavia_unique_dates = [yugoslavia.boardapprovaldate.sort_values().unique()]

dates = np.append(republic_unique_dates, yugoslavia_unique_dates)

In [23]:
# print out the dates that appeared twice in the results
unique_dates, counts = np.unique(dates, return_counts=True)

for i in range(len(unique_dates)) :
    if counts[i] > 1 : 
        print(unique_dates[i])

1983-07-26 00:00:00+00:00
1987-10-13 00:00:00+00:00


And as a final step, try to see if there are any projects in the republics variable and yugoslavia variable that could be the same project.

There are multiple ways to do that. As a suggestion, find unique dates in the republics variable. Then separately find unique dates in the yugoslavia variable. Concatenate (ie append) the results together. And then count the number of times each date occurs in this list. If a date occurs twice, that means the same boardapprovaldate appeared in both the Yugoslavia data as well as in the republics data.

You should find that there are ~~four~~ **two** suspicious cases:

* July 26th, 1983
* ~~March 31st, 1987~~
* October 13th, 1987
* ~~May 23rd, 1989~~

# Conclusion

On July 26th, 1983, for example, projects were approved for Bosnia and Herzegovina, Croatia, Macedonia, Slovenia, and Yugoslavia. The code below shows the projects for that date. You'll notice that Yugoslavia had two projects, one of which was called "Power Transmission Project (03) Energy Managem...". The projects in the other countries were all called "POWER TRANS.III". 

This looks like a case of duplicate data. What you end up doing with this knowledge would depend on the context. For example, if you wanted to get a true count for the total number of projects in the data set, should all of these projects be counted as one project? 

Run the code cell below to see the projects in question.

In [25]:
import datetime

# run this code cell to see the duplicate data
pd.concat([yugoslavia[yugoslavia['boardapprovaldate'] =='1983-07-26'], 
           republics[republics['boardapprovaldate'] == '1983-07-26']])

,regionname,countryname,lendinginstr,totalamt,boardapprovaldate,location,GeoLocID,GeoLocName,Latitude,Longitude,Country,project_name
13046,Europe and Central Asia,Socialist Federal Republic of Yugoslavia,Financial Intermediary Loan,70000000,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,Industrial Credit Project (07)
13047,Europe and Central Asia,Socialist Federal Republic of Yugoslavia,Specific Investment Loan,120000000,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,Power Transmission Project (03) Energy Managem...
13048,Europe and Central Asia,Bosnia and Herzegovina,Specific Investment Loan,0,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,POWER TRANS.III
13050,Europe and Central Asia,Macedonia,Specific Investment Loan,0,1983-07-26 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,POWER TRANS.III
